# Importing and installing required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import re
import math
from sklearn.ensemble import RandomForestRegressor
!pip install biopython
from Bio.SeqUtils.ProtParam import ProteinAnalysis

# Importing data

In [ ]:
data = pd.read_csv('Data.csv')
data.head()

#EDA

In [ ]:
Data.info()

In [ ]:
Data.describe().T

In [ ]:
Data.isnull().sum()

# Data prepration

In [ ]:
Data.drop(['data_source', 'seq_id'], axis = 1, inplace = True)
Data.dropna(inplace = True)

Let's calculate the lenth of protein sequences

In [ ]:
Data['protein_length'] = Data['protein_sequence'].apply(lambda x: len(x))

In [ ]:
amino_count = Data['protein_sequence'].str.split('').explode('protein_sequence').value_counts().drop('')
amino_acids = list(amino_count.index)

In [ ]:
def return_amino_acid_df(df, ac):
  # Normalize each amino acids
  for i in ac:
    df[i]=df['protein_sequence'].str.count(i,re.I)/df['protein_length']
  return df

In [ ]:
Data = return_amino_acid_df(Data, amino_acids)

In [ ]:
# PhysioChemical Properties of Amino acids

#Aromaticity
def calculate_aromaticity(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.aromaticity()

#Molecular Weight
def calculate_molecular_weight(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.molecular_weight()

#Instability Index
def calculate_instability_index(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.instability_index()

#Hydrophobicity
def calculate_hydrophobicity(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.gravy(scale='KyteDoolitle')

#Isoelectric Point
def calculate_isoelectric_point(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.isoelectric_point()

#Charge
def calculate_charge(row):
  sequence = str(row[0])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.charge_at_pH(row[1])

In [ ]:
Data['Aromaticity'] = Data.apply(calculate_aromaticity, axis=1)
Data['Molecular Weight'] = Data.apply(calculate_molecular_weight, axis=1)
Data['Instability Index'] = Data.apply(calculate_instability_index, axis=1)
Data['Hydrophobicity'] = Data.apply(calculate_hydrophobicity, axis=1)
Data['Isoelectric Point'] = Data.apply(calculate_isoelectric_point, axis=1)
Data['Charge'] = Data.apply(calculate_charge, axis=1)

In [ ]:
Data['Aromaticity'] = pd.to_numeric(Data['Aromaticity'])
Data['Molecular Weight'] = pd.to_numeric(Data['Molecular Weight'])
Data['Instability Index'] = pd.to_numeric(Data['Instability Index'])
Data['Hydrophobicity'] = pd.to_numeric(Data['Hydrophobicity'])
Data['Isoelectric Point'] = pd.to_numeric(Data['Isoelectric Point'])
Data['Charge'] = pd.to_numeric(Data['Charge'])

We are done with protein sequence! So just drop it!

In [ ]:
Data.drop('protein_sequence', axis = 1, inplace = True)

# Data spliting

* Training Data = 80% of dataset
* Validation Data = 10% of dataset
* Test Data = 10% of dataset

In [ ]:
train_df, rem_df = train_test_split(Data, train_size=0.8)

val_df, test_df = train_test_split(rem_df, test_size=0.5)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']

# Random Forest

In [ ]:
model = RandomForestRegressor()

## Hyperparameter tuning

In [ ]:
params = {'n_estimators': list(np.arange(60, 510, 10)), 'max_depth': [5, 10, 15, 20, 25, 30, 35, 40]}

HpTRF = GridSearchCV(model, params)

HpTRF.fit(X_train, y_train)

## Model training & prediction

In [ ]:
model = RandomForestRegressor(n_estimators = HpTRF.best_params_['n_estimators'], max_depth = HpTRF.best_params_['max_depth'])

model.fit(X_train, y_train)

y_pred_train_RF = model.predict(X_train)
y_pred_val_RF = model.predict(X_val)
y_pred_test_RF = model.predict(X_test)

## Results

In [ ]:
mse_train_RF = mean_squared_error(y_train, y_pred_train_RF)
mse_val_RF = mean_squared_error(y_val, y_pred_val_RF)
mse_test_RF = mean_squared_error(y_test, y_pred_test_RF)
print("Root Mean Squared error on Train data is:",math.sqrt(mse_train_RF))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val_RF))

In [ ]:
r2_train_RF = r2_score(y_train, y_pred_train_RF)
r2_val_RF = r2_score(y_val, y_pred_val_RF)
r2_test_RF = r2_score(y_test, y_pred_test_RF)
print('r2 score for train is', r2_train_RF)
print('r2 score for val is', r2_val_RF)
print('r2 score for test is', r2_test_RF)

In [ ]:
print('Mean Absolute Error Train:', mean_absolute_error(y_train, y_pred_train_RF))
print('Mean Absolute Error Val:', mean_absolute_error(y_val, y_pred_val_RF))
print('Mean Absolute Error Test:', mean_absolute_error(y_test, y_pred_test_RF))

In [ ]:
sns.residplot(x = y_test, y = y_pred_test_RF)

In [ ]:
sns.regplot(x = y_test, y = y_pred_test_RF)